In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp
import time
import matplotlib.pyplot as plt

# Definicion del problema
Consideramos una serie de objetos cada uno con peso $w_{i}$ y profit $p_{i}$ y una bolsa con capacidad $W$.

El problema de knapsack radica en determinar un subconjunto de $w$ que maximice la suma de los profits llevados, y satisfaga la capacidad de la bolsa.

En otras palabras, ¿que objetos puedo llevar en la bolsa que me maximicen el profit?



In [ ]:
# Creamos una instancia de problema knapsack para trabajar
profits = np.array([18, 15, 10, 10, 18])
weights = np.array([19, 13, 10, 17, 10])
data = dict({'weight': weights,'profit': profits})
num_items = len(profits)
max_weight = int(np.floor(num_items / 2 * np.mean(weights)))
df = pd.DataFrame(data)
print("-------------------------------------")
print("Choose items from: \n ")
print(df)
print(f"with a max weight of: {max_weight}\n ")
print("-------------------------------------")

# Modelado lineal del problema

El problema de optimizacion se puede modelar considerando las variables binarias:
$$
 \begin{equation}
    x_{i}=
    \begin{cases}
      1 & \text{si elemento $i$ es elegido}\  \\
      0 & \text{en otro caso}
    \end{cases}
  \end{equation}$$

Luego el problema queda:


$$ max \sum_{i} p_ix_{i} $$
st
$$\sum_{i} x_{i}w_{i}\leq W$$

En forma matricial, si definimos los vectores $w$, $p$, y $x$ como: 
$$w = [w_1, w_2, \dots , w_n]$$
$$p = [p_1, p_2, \dots , p_n]$$
$$x = [x_1, x_2, \dots , x_n]$$

Donde $w$ y $p$ son parametros, y $x$ es el vector de variables de optimización.

El problema queda:

$$ max\{  p^{t}x\}$$
st
$$ w^{t}x \leq W$$


A partir de ahora, conviene pensar al problema no como hallar los valores de ${x_i}$ óptimos, sino como el problema de hallar el vector $x$ óptimo.

In [ ]:
# Solucion usando cvxpy
def solve_knapsack(profits, weights, max_weight):
  num_items = len(profits)
  x = cp.Variable(num_items, boolean = True) # creacion de variable binaria


  constraint = weights @ x <= max_weight
  profit_loaded = profits @ x

  objective = cp.Maximize(profit_loaded)
  problem = cp.Problem(objective, [constraint])

  # resolucion del problema
  tic = time.perf_counter()
  result = problem.solve(solver=cp.GLPK_MI)
  tac = time.perf_counter()
  time_elapsed = tac-tic
  

  x_opt = x.value
  return x_opt, time_elapsed

In [ ]:
x_opt, time_elapsed = solve_knapsack(profits, weights, max_weight) # no nos interesa retornar el tiempo por ahora
print(f"solution found in: {time_elapsed} seconds")
print("solution: ", x_opt)

# Interpretacion del resultado

El  solver nos da un valor de $x$ optimo, que logra cargar un profit de $43$. Es necesario hacer un post procesamiento para poder mostrar este resultado de forma más agradable, y poder checkear su validez. 

In [ ]:
# calculo del profit cargado
profit_loaded = int(np.sum(profits*x_opt))
# calculo del peso cargado
weight_loaded = int(np.sum(weights*x_opt))

if weight_loaded <= max_weight: # si la solucion es valida

  index_of_chosen = np.where(x_opt == 1)[0] # retorna los indices de los items elegidos.
  output_df = df.iloc[index_of_chosen]
  print("-------------------------------------")
  print("Choosen items are: ")
  print(output_df)
  print(f"The loaded weight is: {weight_loaded} kg, and the maximum was: {max_weight} kg")
  print(f"The loaded profit is:  ${profit_loaded}\n" )
  print("-------------------------------------")

else:
  print("invalid solution, weight exceeded")  


# Algunas consideraciones

> * Si tenemos un problema de Knapsack de $n$ variables (es decir, de $n$ items) entonces el espacio de búsqueda es de tamaño $2^{n}$ posibles vectores $x$ (no todos viables)

> * El problema de optimización es no convexo, y los solvers clásicos utilizan heurísticas y métodos numéricos para resolver el problema.

# Ejercicio:

Dados los items con pesos: $[10, 30, 25, 32, 21, 12, 43]$ y una bolsa con capacidad máxima $73kg$, indicar que items llevar para maximizar la cantidad de items llevados.

Consideración: el resultado debe ser mostrado en un dataframe de pandas, manteniendo los indices originales

In [ ]:
# sol, despues borramos

weights_e = [10, 30,25, 32, 21, 12, 43]
weight_max_e = 73
data = dict({'weights ': weights_e})
df = pd.DataFrame(data)
profits_e = np.ones(len(weights_e))
x_opt_e, time_elapsed_e = solve_knapsack(profits_e, weights_e, weight_max_e)
profit_loaded_e = int(np.sum(profits_e*x_opt_e))
# calculo del peso cargado
weight_loaded_e = int(np.sum(weights_e*x_opt_e))

if weight_loaded_e <= weight_max_e: # si la solucion es valida

  index_of_chosen_e = np.where(x_opt_e == 1)[0] # retorna los indices de los items elegidos.
  output_df_e = df.iloc[index_of_chosen_e]
  print("-------------------------------------")
  print("Choosen items are: ")
  print(output_df)
  print(f"The loaded weight is: {weight_loaded_e} kg, and the maximum was: {weight_max_e} kg")
  print(f"The amount of loaded itemsis:  ${profit_loaded_e}\n" )
  print("-------------------------------------")

else:
  print("invalid solution, weight exceeded")  



# Opcional
Tiempo de ejecucion en funcion de cantidad de items

In [ ]:
max_items = 10000
times = np.array([])
for i in range(5,max_items):
  weights_rand = np.array(abs(np.random.normal(loc= i, scale=i/4, size=i))).astype(int)
  profits_rand = np.array(abs(np.random.normal(loc= i/2, scale=i/4, size=i))).astype(int)
  num_items_rand = len(profits_rand)
  max_weight_rand = int(np.floor(num_items / 2 * np.mean(weights)))
  x_opt, time_elapsed = solve_knapsack(profits_rand, weights_rand, max_weight_rand)
  times = np.append(times,time_elapsed)


times[abs(times - np.mean(times)) < 2 * np.std(times)]; # saco algun outlier
times = times/(10**-3) # lo paso a ms

In [ ]:

def solveNormalEquations(A, b):
  
    
    At = np.transpose(A) # guardo en "At" la traspuesta de A
    inv = np.linalg.inv(At*A) # guardo en "inv" la inversa de A^{t}*A
    x0 = inv*(At*b)  
    
    error = np.linalg.norm(A*x0-b) # Error es la norma 2 de Ax-b
    
    return x0, error

In [ ]:
t = [i for i in range(5, max_items)]
A = np.transpose(np.matrix([t, np.ones(len(t))]))  # A la matriz que tiene como filas t, y una fila de 1, la traspongo.
b = np.transpose(np.array([times])) # vector y traspuesto

xopt, error = solveNormalEquations(A, b)

m = float(xopt[0][0])
n = float(xopt[1][0])

In [ ]:


print(m)
print(n)


In [ ]:
  
plt.title("Tiempo de ejecucion")
plt.xlabel("cantidad de elementos en knapsack")
plt.ylabel("tiempo (ms)")
plt.plot(times, label = "tiempo")
plt.plot(np.multiply(t, m)+n, label = "recta de ajuste")
plt.legend()
plt.show()

Podemos notar un crecimiento de tiempo de ejecucion (esperado) en función. 

Teoricamente, la complejidad del knapsack es de orden $\mathcal{O}(n*W)$

donde $n$ es la cantidad de items y $W$ es la capacidad de la bolsa. 

# Solucion Knapsack con volumenes:

Vamos a usar esta solucion para comparar la validez del codigo creado para Dwave

In [2]:
def solve_knapsack_volumes(profits, weights, max_weight, volumes, max_volume):
  num_items = len(profits)
  x = cp.Variable(num_items, boolean = True) # creacion de variable binaria


  constraint_weights = weights @ x <= max_weight
  constraint_volumes = volumes @ x <= max_volume
  profit_loaded = profits @ x

  objective = cp.Maximize(profit_loaded)
  problem = cp.Problem(objective, [constraint_weights, constraint_volumes])

  # resolucion del problema
  tic = time.perf_counter()
  result = problem.solve(solver=cp.GLPK_MI)
  tac = time.perf_counter()
  time_elapsed = tac-tic
  

  x_opt = x.value
  return x_opt, time_elapsed

In [3]:
# Creamos una instancia de problema knapsack para trabajar
profits = np.array([18, 15, 10, 10, 18])
weights = np.array([19, 13, 10, 17, 10])
#volumes = np.array([12, 17, 14, 15, 11])
volumes = np.array([2, 1, 3, 4, 5])
data = dict({'volumes':volumes, 'weight': weights,'profit': profits})
num_items = len(profits)
max_weight = int(np.floor(num_items / 2 * np.mean(weights)))
max_volume = int(np.floor(num_items / 2 * np.mean(volumes)))
df = pd.DataFrame(data)
print("-------------------------------------")
print("Choose items from: \n ")
print(df)
print(f"with a max weight of: {max_weight}\n ")
print(f"with a max volume of: {max_volume}\n ")
print("-------------------------------------")

-------------------------------------
Choose items from: 
 
   volumes  weight  profit
0        2      19      18
1        1      13      15
2        3      10      10
3        4      17      10
4        5      10      18
with a max weight of: 34
 
with a max volume of: 7
 
-------------------------------------


In [4]:
x_opt, time_elapsed = solve_knapsack_volumes(profits, weights, max_weight, volumes, max_volume) # no nos interesa retornar el tiempo por ahora
print(f"solution found in: {time_elapsed} seconds")
print("solution: ", x_opt)

solution found in: 0.029988500056788325 seconds
solution:  [1. 0. 0. 0. 1.]


In [6]:
# calculo del profit cargado
profit_loaded = int(np.sum(profits*x_opt))
# calculo del peso cargado
weight_loaded = int(np.sum(weights*x_opt))
volume_loaded = int(np.sum(volumes*x_opt))

if weight_loaded <= max_weight and volume_loaded <= max_volume: # si la solucion es valida

  index_of_chosen = np.where(x_opt == 1)[0] # retorna los indices de los items elegidos.
  output_df = df.iloc[index_of_chosen]
  print("-------------------------------------")
  print("Choosen items are: ")
  print(output_df)
  print(f"The loaded weight is: {weight_loaded} kg, and the maximum was: {max_weight} kg")
  print(f"The loaded weight is: {volume_loaded} mt^3, and the maximum was: {max_volume} mt^3")
  print(f"The loaded profit is:  ${profit_loaded}\n" )
  print("-------------------------------------")

else:
  print("invalid solution, weight exceeded")

-------------------------------------
Choosen items are: 
   volumes  weight  profit
0        2      19      18
4        5      10      18
The loaded weight is: 29 kg, and the maximum was: 34 kg
The loaded weight is: 7 mt^3, and the maximum was: 7 mt^3
The loaded profit is:  $36

-------------------------------------
